# Modify Nodes in an NDEx Network Based on a TSV File

In this notebook, you will modify a network from NDEx that shows the interactions between the NCI ITCR projects.

This version is missing some node attributes and needs corrections to the organization of their attributes.

The network will be updated based on a TSV file where each row has information about one of the ITCR projects.

The workflow:
 * Load the TSV file as a Pandas Dataframe
 * Create an NiceCX network from the network on NDEx
 * For each row in the Dataframe:
  * Identify the corresponding node (skip if there is no such node)
  * Add attributes to the node based on the following columns:
   * “Category”
   * “Tool URL”
   * “Code Repo”
   * “Project Info” 
  * Set the "Tool Description" attribute of the node to the value of the "Body" column
  * Set the node name to be the value of the "Title" column
  * Set the "label" attribute to the value of the "Tool Acronym" when available, otherwise use the value of the "Title" column.
 * Save the network to your account on NDEx

The final network should still have 36 nodes/81 edges but provide extra info and link out to external resource

First, import the ndex2 Python client and Pandas

In [1]:

import ndex2
import pandas as pd

Next, set a variable to hold the UUID of the network we want to modify

In [2]:
my_network_uuid = '04c0a7e8-af92-11e7-94d3-0ac135e8bacf'

Set variables to hold your account and password 

In [3]:
my_account = "drh"
my_password = "drh"

Create a NiceCX network from the ITCR network..

In [4]:
print("downloading network and buiding NiceCX...")
itcr_network = ndex2.create_nice_cx_from_server(server='public.ndexbio.org', uuid=my_network_uuid)
#my_network = ndex2.create_nice_cx_from_server(server='public.ndexbio.org', uuid=my_network_uuid, account=my_account, password = my_password)
print("done")
print(itcr_network.get_summary())

downloading network and buiding NiceCX...
done
Name: ITCR Connectivity Map
Nodes: 36
Edges: 81
Node Attributes: 0
Edge Attributes: 241



Load the TSV to a Datframe

In [5]:
tsv_path = '../resources/itcr-conn-map-v2.0_nodes.txt'
node_data = pd.read_csv(tsv_path, sep="\t")
node_data

,Title,Tool Acronym,Body,Category,Tool URL,Code Repo,Project Info
0,3D Slicer,NaN,3D Slicer is the free open source software for...,Imaging,https://www.slicer.org,https://github.com/Slicer,https://projectreporter.nih.gov/project_info_d...
1,Allele-Specific Alternative mRNA processing (A...,ASARP,A software pipeline for prediction of allele-s...,omics,https://www.ibp.ucla.edu/research/xiao/Softwar...,https://github.com/cyruschan/ASARP,https://projectreporter.nih.gov/project_info_d...
2,Apache Clinical Text and Knowledge Extraction ...,cTAKES,The tool extracts deep phenotypic information ...,Clinical,http://ctakes.apache.org,http://ctakes.apache.org/downloads.cgi,https://projectreporter.nih.gov/project_info_d...
3,Bioconductor,NaN,Bioconductor provides tools for the analysis a...,Omics,https://www.bioconductor.org,http://bioconductor.org/developers/how-to/git/,https://projectreporter.nih.gov/project_info_d...
4,Cancer and Phenomics Toolkit (CaPTK),CaPTK,CaPTk is a tool that facilitates translation o...,Imaging,http://captk.projects.nitrc.org,NaN,https://projectreporter.nih.gov/project_info_d...
5,Cancer Slide Digital Archive (CDSA),CDSA,The CDSA is a web-based platform to support th...,Imaging,http://cancer.digitalslidearchive.net,NaN,https://projectreporter.nih.gov/project_info_d...
6,Cancer-Related Analysis of Variants Toolkit (C...,CRAVAT,CRAVAT is an easy to use web-based tool for an...,Omics,http://www.cravat.us/CRAVAT/,https://github.com/KarchinLab/CRAVAT,https://projectreporter.nih.gov/project_info_d...
7,Cistrome,NaN,Curated and processed human/mouse ChIP/DNase-s...,Omics,http://cistrome.org/Cistrome/Cistrome_Project....,NaN,https://projectreporter.nih.gov/project_info_d...
8,Clinical Interpretation of Variants in Cancer ...,CIViC,"CIViC is an open access, open source, communit...","Omics, Clinical",https://civic.genome.wustl.edu/home,https://github.com/griffithlab,https://projectreporter.nih.gov/project_info_d...
9,Cores Of Recurrent Events (CORE),CORE,CORE is a statistically supported computationa...,Omics,NaN,http://cran.us.r-project.org/web/packages/CORE/,https://projectreporter.nih.gov/project_info_d...


Annotate the nodes in the network according to the plan.


In [16]:
label_to_node_map = {}
for index, node in itcr_network.get_nodes():
    label = itcr_network.get_node_attribute(node, "label")
    if label:
        label_to_node_map[label] = node
        
for index, row in node_data.iterrows():
    # Identify the corresponding node (skip if there is no such node)
    node = label_to_node_map.get(row['Title'])
    if node:
        
        # Add attributes to the node
        attributes = ["Category", "Tool URL", "Code Repo", "Project Info"]
        for att in attributes:
            itcr_network.set_node_attribute(node, att, row[att])
            
        # Set the "Tool Description" attribute of the node to the value of the "Body" column
        itcr_network.set_node_attribute(node, "Tool Description", row['Body'])
        
        # Set the node name to be the value of the "Title" column
        node.set_name(row["Title"])
        
        # Set the "label" attribute to the value of the "Tool Acronym" when available, 
        # otherwise use the value of the "Title" column.
        acronym = row["Tool Acronym"]
        if acronym is "NaN":
            acronym = None
        itcr_network.set_node_attribute(node, "label", acronym or row["Title"])


{"r": "C-BIBOP", "@id": 1, "n": "C-BIBOP"}
{"r": "3D Slicer", "@id": 2, "n": "3D Slicer"}
{"r": "CaPTK", "@id": 3, "n": "CaPTK"}
{"r": "DICOM ToolKit (DCMTK)", "@id": 4, "n": "DICOM ToolKit (DCMTK)"}
{"r": "LesionTracker", "@id": 5, "n": "LesionTracker"}
{"r": "PETCT-CoSeg", "@id": 6, "n": "PETCT-CoSeg"}
{"r": "QUIP", "@id": 7, "n": "QUIP"}
{"r": "SlicerDMRI", "@id": 8, "n": "SlicerDMRI"}
{"r": "XNAT", "@id": 9, "n": "XNAT"}
{"r": "GenePattern", "@id": 10, "n": "GenePattern"}
{"r": "AMARETTO", "@id": 11, "n": "AMARETTO"}
{"r": "MuPIT", "@id": 12, "n": "MuPIT"}
{"r": "BMEG", "@id": 13, "n": "BMEG"}
{"r": "NG-CHM", "@id": 14, "n": "NG-CHM"}
{"r": "CDSA", "@id": 15, "n": "CDSA"}
{"r": "dcmqi", "@id": 16, "n": "dcmqi"}
{"r": "RADIOMICS.io", "@id": 17, "n": "RADIOMICS.io"}
{"r": "Bioconductor", "@id": 18, "n": "Bioconductor"}
{"r": "cBioPortal", "@id": 19, "n": "cBioPortal"}
{"r": "NDEx", "@id": 20, "n": "NDEx"}
{"r": "CIViC", "@id": 21, "n": "CIViC"}
{"r": "HIVE (BioMuta)", "@id": 22, "n":

Spot-check the network by printing out a sample of the nodes in the modified network

In [15]:
print(itcr_network.get_nodes())

[(1, <nicecxModel.cx.aspects.NodeElement.NodeElement object at 0x11f710310>), (2, <nicecxModel.cx.aspects.NodeElement.NodeElement object at 0x11f710050>), (3, <nicecxModel.cx.aspects.NodeElement.NodeElement object at 0x11f710290>), (4, <nicecxModel.cx.aspects.NodeElement.NodeElement object at 0x11f710250>), (5, <nicecxModel.cx.aspects.NodeElement.NodeElement object at 0x11f710150>), (6, <nicecxModel.cx.aspects.NodeElement.NodeElement object at 0x11f7102d0>), (7, <nicecxModel.cx.aspects.NodeElement.NodeElement object at 0x11f710190>), (8, <nicecxModel.cx.aspects.NodeElement.NodeElement object at 0x11f7103d0>), (9, <nicecxModel.cx.aspects.NodeElement.NodeElement object at 0x11f710090>), (10, <nicecxModel.cx.aspects.NodeElement.NodeElement object at 0x11f6eb850>), (11, <nicecxModel.cx.aspects.NodeElement.NodeElement object at 0x11f6ebf90>), (12, <nicecxModel.cx.aspects.NodeElement.NodeElement object at 0x11f6ebbd0>), (13, <nicecxModel.cx.aspects.NodeElement.NodeElement object at 0x11f6eb5

In [12]:
counter = 0
for id, node in itcr_network.get_nodes():
    if counter > 3:
        break;
    print(node.get_name())
    print(itcr_network.get_node_attribute(node, "label"))
    print(itcr_network.get_node_attribute(node, "Tool Description"))
    print("----")
print(itcr_network)

C-BIBOP
None
None
----
3D Slicer
None
None
----
CaPTK
None
None
----
DICOM ToolKit (DCMTK)
None
None
----
LesionTracker
None
None
----
PETCT-CoSeg
None
None
----
QUIP
None
None
----
SlicerDMRI
None
None
----
XNAT
None
None
----
GenePattern
None
None
----
AMARETTO
None
None
----
MuPIT
None
None
----
BMEG
None
None
----
NG-CHM
None
None
----
CDSA
None
None
----
dcmqi
None
None
----
RADIOMICS.io
None
None
----
Bioconductor
None
None
----
cBioPortal
None
None
----
NDEx
None
None
----
CIViC
None
None
----
HIVE (BioMuta)
None
None
----
PDX Finder
None
None
----
Trinity
None
None
----
FHIR ShEx
None
None
----
CLAMP-Cancer
None
None
----
Galaxy-P multi-omics
None
None
----
CRAVAT
None
None
----
UCSC Xena
None
None
----
Integrative Genomics Viewer (IGV)
None
None
----
THRIVE
None
None
----
OmniSearch
None
None
----
TCIA
None
None
----
HIVE (BioExpress)
None
None
----
TCPA
None
None
----
TIES
None
None
----
[{"numberVerification": [{"longNumber": 281474976710655}]}, {"metaData": [{"elementCount"

Upload the network to your NDEx account as a new network.

If you started by loading an one of your networks (such as a clone of the example network), then you can *update* it instead, overwriting its content. 

In the commented line of below, the optional parameter *update_uuid* specifies the update behavior.

In [ ]:
upload_message = my_network.upload(my_server, my_account, my_password)
# upload_message = my_network.upload(my_server, my_account, my_password, update_uuid=my_network_uuid)
print(upload_message)